<a href="https://colab.research.google.com/github/Daniel-Pereira3264/Machine-Learning-using-Python-Edge-Training/blob/main/Week10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 19 - Logistic Regression and Lasso Regression

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression

from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

Import Dataset

In [ ]:
import pandas as pd
df = pd.read_csv('/content/diabetes - diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# normalization

data = df.iloc[:, :-1]
# z score normalization
scaler = zscore(data)

# min max normalization
min_max=MinMaxScaler()
#standard=StandardScaler()
scaled_min_max = min_max.fit_transform(data)


normalized_data = pd.DataFrame(scaled_min_max)
normalized_data[:2]

,0,1,2,3,4,5,6,7
0,0.352941,0.743719,0.590164,0.353535,0.0,0.500745,0.234415,0.483333
1,0.058824,0.427136,0.540984,0.292929,0.0,0.396423,0.116567,0.166667


In [ ]:
X=df.iloc[:,:-1]
y=df['Outcome']

In [ ]:
X_train, x_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
# without normalization
lr_model = LogisticRegression(max_iter=200)
lr_model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = lr_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy:  {100*accuracy : .4f}')

Accuracy:   74.6753


In [ ]:
from sklearn.model_selection import GridSearchCV

X =df.iloc[:, :-1]

y=df['Outcome']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr_model = LogisticRegression()

param_grid = {
    'max_iter': [1000, 2000, 3000, 4000, 5000]
    #'penalty': ['l1', 'l2', 'elasticnet', None],
    #'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
}

grid_search = GridSearchCV(estimator=lr_model, param_grid=param_grid, cv=5, n_jobs=1)

grid_search.fit(X_train, y_train)
print("Grid Search Best Parameters:", grid_search.best_params_)
print("Grid Search Best Score:", grid_search.best_score_)


Grid Search Best Parameters: {'max_iter': 1000}
Grid Search Best Score: 0.7655071304811409


In [ ]:
# feature selection using corelation coefficient
corr_matrix =df.corr()['Outcome'][:]
print(corr_matrix.sort_values(ascending=False))

Outcome                     1.000000
Glucose                     0.466581
BMI                         0.292695
Age                         0.238356
Pregnancies                 0.221898
DiabetesPedigreeFunction    0.173844
Insulin                     0.130548
SkinThickness               0.074752
BloodPressure               0.065068
Name: Outcome, dtype: float64


In [ ]:
df.columns.tolist()

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [ ]:
# using chi-square
# chi-square feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
from numpy import array

x=df.iloc[:,:-1]
y=df["Outcome"]

select = SelectKBest(score_func=chi2, k=3)
z = select.fit_transform(x,y)

filter = select.get_support()

feat_labels=df.columns.tolist()

features = array(feat_labels)
print("All features:")
print(features)

print("Selected best 5:")
print(filter)

All features:
['Pregnancies' 'Glucose' 'BloodPressure' 'SkinThickness' 'Insulin' 'BMI'
 'DiabetesPedigreeFunction' 'Age' 'Outcome']
Selected best 5:
[False  True False False  True False False  True]


In [ ]:
# classification based on after chi-square feature selection
X=df[['Glucose','Insulin', 'Age']]
y=df['Outcome']

X_train, x_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

# without normalization
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = lr_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {100*accuracy:.4f}')


Accuracy: 75.9740


# Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load dataset
df_fuel =pd.read_csv('/content/FuelConsumptionCo2 - FuelConsumptionCo2.csv')

X =df_fuel[['FUELCONSUMPTION_COMB', 'FUELCONSUMPTION_COMB_MPG']]
y =df_fuel['CO2EMISSIONS']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lasso Regression model
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

# Predictions and evaluation
y_pred = lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 630.2924254849524


In [ ]:
# lasso as feature selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
import numpy as np

# feature and target
X = df.iloc[:,:-1]
y = df['Outcome']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Lasso regression model
lasso = Lasso(alpha=0.1)  # Adjust alpha as needed

# Fit the model to the training data
lasso.fit(X_train, y_train)

# Get feature importance (coefficients) from the model
feature_importance = lasso.coef_

# Get indices of top most features
top_feature_indices = np.argsort(feature_importance)[::-1][:5]  # Get top 5 features


feature_names = df.iloc[:,:-1].columns # get efature names

# Print top most features with names
print("Top 5 most important features:")
for i, idx in enumerate(top_feature_indices):
    print(f"Feature {i+1}: {feature_names[idx]} - Importance: {feature_importance[idx]}")

Top 5 most important features:
Feature 1: BMI - Importance: 0.01355058013699686
Feature 2: Age - Importance: 0.0070836007622608384
Feature 3: Glucose - Importance: 0.005748636345242389
Feature 4: SkinThickness - Importance: 0.0004268420026928431
Feature 5: DiabetesPedigreeFunction - Importance: 0.0


In [ ]:
# classification based on after lasso feature selection
X=df[['BMI','Age','Glucose']]
y=df['Outcome']

X_train, x_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

# without normalization
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = lr_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7467532467532467


In [ ]:
# classification based on after lasso feature selection
X=df[['BMI','Age','Glucose']]
y=df['Outcome']

X_train, x_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

# without normalization


# Predictions and evaluation
y_pred = lr_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.7467532467532467


# SVM

In [ ]:
# classification based on after lasso feature selection
from sklearn.svm import SVC

X=df[['BMI','Age','Glucose']]
y=df['Outcome']

X_train, x_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

# without normalization
svm=SVC()
svm.fit(X_train, y_train)

# Predictions and evaluation
y_pred = svm.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {100*accuracy}')

Accuracy: 77.27272727272727
